

1. **Thread Pools**
2. **Main Thread**
3. **Thread Priority**
4. **Daemon Threads**
5. **Synchronizing Threads**


---

# **1. Python – Thread Pools**

---

## **Introduction**

When you have many small tasks to execute concurrently, creating a new thread for each task can be inefficient.
Thread creation and destruction involve overhead. To manage this efficiently, Python provides **Thread Pools** through the `concurrent.futures` module.

A **Thread Pool** maintains a fixed number of worker threads.
Tasks are submitted to the pool, and available threads execute them.
This approach is highly efficient for handling a large number of short-lived tasks.

---

## **Using ThreadPoolExecutor**

`ThreadPoolExecutor` is part of the `concurrent.futures` module and provides a high-level interface for managing threads.

---

### **Example 1: Simple Thread Pool**

```python
from concurrent.futures import ThreadPoolExecutor
import time

def worker(task_id):
    print(f"Task {task_id} started.")
    time.sleep(2)
    print(f"Task {task_id} completed.")

# Creating a thread pool with 3 workers
with ThreadPoolExecutor(max_workers=3) as executor:
    for i in range(5):
        executor.submit(worker, i)

print("All tasks submitted.")
```

**Explanation:**

* `ThreadPoolExecutor(max_workers=3)` creates a pool of 3 threads.
* `submit()` adds tasks to the pool for execution.
* The pool automatically manages which task is executed by which thread.

---

### **Example 2: Returning Results from Threads**

```python
from concurrent.futures import ThreadPoolExecutor
import time

def square(n):
    time.sleep(1)
    return n * n

with ThreadPoolExecutor(max_workers=3) as executor:
    results = list(executor.map(square, [1, 2, 3, 4, 5]))

print("Results:", results)
```

**Explanation:**

* `executor.map()` automatically distributes work to the available threads.
* It collects the results as they complete.

---

### **Advantages of Thread Pools**

1. Reduces thread creation overhead.
2. Manages a fixed number of threads efficiently.
3. Automatically queues tasks until threads are free.
4. Simplifies code for large sets of concurrent tasks.

---

# **2. Python – Main Thread**

---

## **Introduction**

Every Python program starts with a single thread called the **main thread**.
This main thread is created automatically by the Python interpreter and executes the main program.
When you create new threads, they become **child threads** of the main thread.

---

## **Main Thread Characteristics**

1. The main thread controls the program’s execution flow.
2. It can create, start, and manage other threads.
3. If the main thread terminates before child threads finish, non-daemon threads continue running until completion.
4. You can obtain the main thread object using `threading.main_thread()`.

---

### **Example 1: Getting the Main Thread**

```python
import threading
import time

def worker():
    print(f"Worker thread: {threading.current_thread().name}")

print("Main thread:", threading.main_thread().name)

t1 = threading.Thread(target=worker, name="Worker-1")
t1.start()
t1.join()

print("Main thread finished.")
```

**Explanation:**

* The `main_thread()` function returns the main thread instance.
* The `current_thread()` function returns the thread currently executing.

---

### **Example 2: Main Thread Waiting for Others**

```python
import threading
import time

def task():
    print("Child thread running")
    time.sleep(2)
    print("Child thread finished")

threads = []
for i in range(3):
    t = threading.Thread(target=task)
    threads.append(t)
    t.start()

# Main thread waits for all threads to finish
for t in threads:
    t.join()

print("Main thread exiting after all child threads are done.")
```

---

# **3. Python – Thread Priority**

---

## **Introduction**

In many operating systems, threads can have priorities that affect their scheduling.
Higher-priority threads may receive more CPU time than lower-priority threads.

However, **Python’s threading module does not provide direct control over thread priority**.
Thread scheduling and priority are managed entirely by the **operating system**.

---

## **Workarounds for Thread Priority**

Although you cannot set explicit priorities, you can **influence** thread execution order using:

1. `time.sleep()` – make certain threads yield CPU time.
2. Task ordering or controlled sequencing.
3. Using the `queue.PriorityQueue` class to manage task priorities, not thread priorities.

---

### **Example: Simulating Thread Priority Using Priority Queue**

```python
import threading
import queue
import time

def worker(priority, name):
    time.sleep(1)
    print(f"Task {name} with priority {priority} completed")

# Creating a priority queue
q = queue.PriorityQueue()

# Adding tasks with different priorities
q.put((1, "High"))
q.put((3, "Low"))
q.put((2, "Medium"))

def process_tasks():
    while not q.empty():
        priority, name = q.get()
        worker(priority, name)
        q.task_done()

t = threading.Thread(target=process_tasks)
t.start()
t.join()
```

**Explanation:**

* Tasks with lower priority numbers (1 = highest priority) are processed first.
* This simulates priority handling at the task level.

---

# **4. Python – Daemon Threads**

---

## **Introduction**

A **daemon thread** is a background thread that runs continuously but does not prevent the program from exiting.
When all non-daemon (main or worker) threads finish, daemon threads are terminated automatically by Python.

Typical examples include:

* Background monitoring
* Logging services
* Auto-saving mechanisms

---

## **Creating a Daemon Thread**

You can mark a thread as a daemon in two ways:

1. By setting `daemon=True` when creating the thread.
2. By setting `t.daemon = True` before calling `start()`.

---

### **Example 1: Daemon Thread**

```python
import threading
import time

def background_task():
    while True:
        print("Daemon thread running...")
        time.sleep(1)

daemon_thread = threading.Thread(target=background_task, daemon=True)
daemon_thread.start()

time.sleep(3)
print("Main thread finished. Daemon thread will terminate automatically.")
```

**Explanation:**

* The daemon thread runs in the background, but when the main thread finishes, the daemon thread is killed automatically.

---

### **Example 2: Non-Daemon Thread (for comparison)**

```python
import threading
import time

def worker():
    print("Worker thread starting")
    time.sleep(5)
    print("Worker thread finished")

t = threading.Thread(target=worker)
t.start()
print("Main thread exiting... but will wait for worker.")
```

**Explanation:**

* Non-daemon threads block the program from exiting until they complete.

---

## **Key Difference**

| Thread Type       | Behavior on Program Exit                       |
| ----------------- | ---------------------------------------------- |
| Non-Daemon Thread | The program waits for the thread to complete   |
| Daemon Thread     | Automatically terminated when main thread ends |

---

# **5. Python – Synchronizing Threads**

---

## **Introduction**

When multiple threads access shared data simultaneously, data inconsistency can occur.
For example, two threads may try to update the same variable at the same time.
To prevent this, **thread synchronization** techniques are used.

Python provides synchronization primitives like:

* `Lock`
* `RLock` (reentrant lock)
* `Semaphore`
* `Condition`
* `Event`

---

## **1. Using Lock**

A **Lock** ensures that only one thread can access a shared resource at a time.

### **Example: Using Lock**

```python
import threading
import time

lock = threading.Lock()
counter = 0

def increment():
    global counter
    for _ in range(100000):
        with lock:
            counter += 1

threads = []
for _ in range(5):
    t = threading.Thread(target=increment)
    threads.append(t)
    t.start()

for t in threads:
    t.join()

print("Final counter value:", counter)
```

**Explanation:**

* Only one thread at a time can execute the code inside the `with lock:` block.
* Without locks, race conditions may cause incorrect results.

---

## **2. Using RLock (Reentrant Lock)**

An `RLock` allows a thread to acquire the same lock multiple times.

```python
import threading

lock = threading.RLock()

def recursive_function(n):
    with lock:
        if n > 0:
            print(f"Lock acquired by {threading.current_thread().name}, n={n}")
            recursive_function(n-1)

t = threading.Thread(target=recursive_function, args=(3,))
t.start()
t.join()
```

**Explanation:**

* A normal `Lock` would deadlock if a thread tried to acquire it again.
* `RLock` allows recursive acquisition by the same thread.

---

## **3. Using Semaphore**

A **Semaphore** allows a limited number of threads to access a resource simultaneously.

```python
import threading
import time

semaphore = threading.Semaphore(2)

def access_resource(i):
    with semaphore:
        print(f"Thread {i} accessing resource")
        time.sleep(2)
        print(f"Thread {i} done")

threads = []
for i in range(5):
    t = threading.Thread(target=access_resource, args=(i,))
    threads.append(t)
    t.start()

for t in threads:
    t.join()
```

**Explanation:**

* Only two threads at a time can access the resource (due to `Semaphore(2)`).

---

## **4. Using Condition**

A **Condition** allows threads to wait for a certain condition before proceeding.

```python
import threading
import time

condition = threading.Condition()
ready = False

def waiter():
    with condition:
        print("Waiting for signal...")
        condition.wait()
        print("Received signal. Continuing.")

def notifier():
    global ready
    time.sleep(2)
    with condition:
        ready = True
        print("Sending signal...")
        condition.notify()

t1 = threading.Thread(target=waiter)
t2 = threading.Thread(target=notifier)

t1.start()
t2.start()

t1.join()
t2.join()
```

**Explanation:**

* `wait()` makes the thread pause until it receives a `notify()` signal.
* Useful for thread communication.

---

## **5. Using Event**

An **Event** is a simple mechanism to signal between threads.

```python
import threading
import time

event = threading.Event()

def worker():
    print("Worker waiting for event to start...")
    event.wait()
    print("Worker received event signal.")

def controller():
    time.sleep(3)
    print("Controller setting event.")
    event.set()

t1 = threading.Thread(target=worker)
t2 = threading.Thread(target=controller)

t1.start()
t2.start()

t1.join()
t2.join()
```

**Explanation:**

* `event.wait()` pauses until `event.set()` is called.
* It provides simple thread communication.

---

# **Summary Table**

| Synchronization Tool | Purpose                                              |
| -------------------- | ---------------------------------------------------- |
| `Lock`               | Prevents simultaneous access to a shared resource    |
| `RLock`              | Allows same thread to acquire lock multiple times    |
| `Semaphore`          | Limits number of threads accessing a resource        |
| `Condition`          | Thread waits for certain condition before proceeding |
| `Event`              | Used for signaling between threads                   |

---

